***Busqueda de N1 min y max***
$$\int_\Omega \nabla u \cdot \nabla v~dx = \int_\Omega fv~dx$$
$$B_{max} = \frac{1}{2}\frac{\int V_L(t)dt}{NA} + \frac{NI_{DC}}{RA} < B_{sat}$$
$$\frac{1}{2}\frac{V_dDT_s}{NA} + \frac{NI_{DC}}{RA} < B_{sat}$$

Con

$$R = \frac{l_e}{\mu_0 \mu_e A}$$

Desmos: https://www.desmos.com/calculator/x80nw1l7ab

Cosas de TME:
$$N_{min} = \frac{1}{2}\frac{V_dDT_s}{B_{sat}A_{min}}$$

In [1]:
from scipy.optimize import fsolve
import numpy as np

def get_turns(Vd, Vo2, Vo3, D_mean, N1, Al):
    L1 = Al*N1**2
    N2 = N1*(Vo2/Vd)*(1-D_mean)/(D_mean)
    L2 = Al*N2**2
    N3 = N1*Vo3/Vd*(1-D_mean)/(D_mean)
    L3 = Al*N3**2
    print("")
    print("\tN1 = ", N1, " vueltas")
    print("\tL1 = ", L1*1E3, "mHy")
    print("\tN2 = ", N2, " vueltas")
    print("\tL2 = ", L2*1E3, "mHy")
    print("\tN3 = ", N3, " vueltas")
    print("\tL3 = ", L3*1E3, "mHy")
    print("")
    print("\tNormalizando:")
    N2 = np.round(N2)
    L2 = Al*N2**2
    N3 = np.round(N3)
    L3 = Al*N3**2
    print("\tN1 = ", N1, " vueltas")
    print("\tL1 = ", L1*1E3, "mHy")
    print("\tN2 = ", N2, " vueltas")
    print("\tL2 = ", L2*1E3, "mHy")
    print("\tN3 = ", N3, " vueltas")
    print("\tL3 = ", L3*1E3, "mHy")
    print("")
    Vo2 = Vd*(N2/N1)*(D_mean/(1-D_mean))
    Vo3 = Vd*(N3/N1)*(D_mean/(1-D_mean))
    print("\tVo2 = ", Vo2, "V")
    print("\tVo3 = ", Vo3, "V")
    print("")
    return

Bsat = 220E-3   #En realidad la datasheet dice 320mT pero Miguel dice que mas de 220mT no le sacamos

#Datos
Vd = 280
#Ponemos un D alto para asegurarnos que ni asi va a saturar
D = 0.45    #Maximo conservador para el modulo pwm
Fs = 75E3
Ts = 1/Fs
#Corriente DC maxima para el MOS
IDC = 2
#Datos del nucleo
Al = 5200E-9
Al_min = Al*(1-0.2)
Al_max = Al*(1+0.3)
A_min = 229E-6
le = 97E-3
u0 = 4*np.pi*1E-7
ue = 1690
Ae = 234E-6
R = le/(u0*ue*A_min)

#Ahora encontramos los parametros limite y deseados
N_min = 0.5*Vd*D*Ts/(Bsat*A_min)
print("N_min = ", N_min, " vueltas. Redondeamos a ", np.ceil(N_min))

#Este es el N_min para que el nucleo no sature por su propia inductancia, ahora tenemos que ver un Idc maximo admitible
#Resolvemos la ecuacion cuadratica: 0.5*Vd*D*Ts/(A_min) + (N**2)*IDC/(R*A_min) - Bsat*N = 0
coeffs = [IDC*Al/(A_min), - Bsat, 0.5*Vd*D*Ts/(A_min)]
print("Raices:", np.roots(coeffs), ",\t Dan imaginarias, asi que la IDC es muy grande, probamos con algo mas bajo")
#Vemos que nos da raices complejas, asi que la IDC (que es el unico parametro que podemos mover, es muy alta)

#Resto de parametros del transformador
Vo2 = 24+1 #+1Vdc para compensar caida en diodos
Vo3 = 16+1


print("Distintas opciones de vueltas para distintos rangos de IDC:\n")
print("Opcion 1:\n")
IDC = 100E-3
coeffs = [IDC/(R*A_min), - Bsat, 0.5*Vd*D*Ts/(A_min)]
print("\tRaices:", np.roots(coeffs), ",\t Con IDC = ", IDC*1E3, "mA")
N1 = 60
get_turns(Vd, Vo2, Vo3, 0.35, N1, Al)

N_min =  16.6732830488289  vueltas. Redondeamos a  17.0
Raices: [2.42211538+8.65462812j 2.42211538-8.65462812j] ,	 Dan imaginarias, asi que la IDC es muy grande, probamos con algo mas bajo
Distintas opciones de vueltas para distintos rangos de IDC:

Opcion 1:

	Raices: [79.37744072 21.10677483] ,	 Con IDC =  100.0 mA

	N1 =  60  vueltas
	L1 =  18.72 mHy
	N2 =  9.948979591836737  vueltas
	L2 =  0.5147074135776762 mHy
	N3 =  6.76530612244898  vueltas
	L3 =  0.23800070803831738 mHy

	Normalizando:
	N1 =  60  vueltas
	L1 =  18.72 mHy
	N2 =  10.0  vueltas
	L2 =  0.52 mHy
	N3 =  7.0  vueltas
	L3 =  0.2548 mHy

	Vo2 =  25.128205128205124 V
	Vo3 =  17.589743589743588 V



In [5]:
##Escribir todas las posibilidades de N1 vueltas a un xml
import pandas as pd

def save_turns(Vd, Vo2, Vo3, D_mean, N1, Al, dataframe):
    L1 = Al*N1**2
    N2 = N1*(Vo2/Vd)*(1-D_mean)/(D_mean)
    L2 = Al*N2**2
    N3 = N1*Vo3/Vd*(1-D_mean)/(D_mean)
    L3 = Al*N3**2

    N2_norm = np.round(N2)
    L2_norm = Al*N2**2
    N3_norm = np.round(N3)
    L3_norm = Al*N3**2

    Vo2_norm = Vd*(N2_norm/N1)*(D_mean/(1-D_mean))
    Vo3_norm = Vd*(N3_norm/N1)*(D_mean/(1-D_mean))
    # add row to the DataFrame
    dataframe.loc[len(dataframe)] = [Vd, Vo2, Vo3, Al*1E9, N1, L1*1E3, N2, L2*1E3, N3, L3*1E3, ' ', N2_norm, L2_norm*1E3, N3_norm, L3_norm*1E3, Vo2_norm, Vo3_norm]
    return dataframe


Al = 5200E-9
# create an empty DataFrame
df = pd.DataFrame(columns=['Vd', 'Vo2', 'Vo3', 'AL[nHy]', 'N1', 'L1[mHy]', 'N2', 'L2[mHy]', 'N3', 'L3[mHy]', 'Normalizamos...', 'N2_norm', 'L2_norm[mHy]', 'N3_norm', 'L3_norm[mHy]', 'Vo2_norm', 'Vo3_norm'])

# add some rows to the DataFrame

# create an XML file from the DataFrame
for N1 in range(25, 71, 1):
    df = save_turns(Vd, Vo2, Vo3, 0.30, N1, Al, df)

print(df.to_string(index=False, max_colwidth=20, float_format='%.3f'))

 Vd  Vo2  Vo3  AL[nHy]  N1  L1[mHy]     N2  L2[mHy]    N3  L3[mHy] Normalizamos...  N2_norm  L2_norm[mHy]  N3_norm  L3_norm[mHy]  Vo2_norm  Vo3_norm
280   25   17 5200.000  25    3.250  5.208    0.141 3.542    0.065                    5.000         0.141    4.000         0.065    24.000    19.200
280   25   17 5200.000  26    3.515  5.417    0.153 3.683    0.071                    5.000         0.153    4.000         0.071    23.077    18.462
280   25   17 5200.000  27    3.791  5.625    0.165 3.825    0.076                    6.000         0.165    4.000         0.076    26.667    17.778
280   25   17 5200.000  28    4.077  5.833    0.177 3.967    0.082                    6.000         0.177    4.000         0.082    25.714    17.143
280   25   17 5200.000  29    4.373  6.042    0.190 4.108    0.088                    6.000         0.190    4.000         0.088    24.828    16.552
280   25   17 5200.000  30    4.680  6.250    0.203 4.250    0.094                    6.000         0.203 

In [6]:
#Corrientes en el trafo
Bsat = 220E-3   #En realidad la datasheet dice 320mT pero Miguel dice que mas de 220mT no le sacamos

#Datos
Vd = 300
#Ponemos un D alto para asegurarnos que ni asi va a saturar
D = 0.5    #Maximo conservador para el modulo pwm
Fs = 75E3
Ts = 1/Fs
Al = 5200E-9
N1 = 62
N2 = 13
N3 = 9
D_max = 0.41
IN1_DC_max = (A_min*R/N1)*(Bsat-0.5*Vd*D_max*Ts/(N1*A_min))
print("IN1_DC_max = ", IN1_DC_max*1E3, "mA")

IN2_DC_max = IN1_DC_max*N1/N2
IN3_DC_max = IN1_DC_max*N1/N3

print("IN2_DC_max = ", IN2_DC_max*1E3, "mA")
print("IN3_DC_max = ", IN3_DC_max*1E3, "mA")

IN1_DC_max =  119.52419652872707 mA
IN2_DC_max =  570.0384757523907 mA
IN3_DC_max =  823.3889094201198 mA


***Snubber***
$$V_{DS} > V_{C_{SN}} > V_d + V_{o}\frac{N1}{N2}$$

$$C_{snubber} > \frac{L_{disp}*I_{L_{disp}}}{V_{sw_{max}^2 - (V_{d} + V_o \frac{N1}{N2})^2}}$$

$$R_{snubber} < \frac{T_{ON_{min}}}{3C_{snubber}} = \frac{D_{min}Ts}{3C_{snubber}}$$

In [9]:
#Estimacion de Ldisp REVISAR
L1 = Al*N1**2
Ldisp = 0.1*L1
ILdisp = IN1_DC_max+Vd*0.5*Ts/Ldisp

Vsw_max = 900

Csn_min = Ldisp*ILdisp/(Vsw_max**2 - (Vd+Vo2*N1/N2)**2)
print("Csn_min = ", Csn_min*1E9, "nF")
Csn = 4.7E-9
print("Csn = ", Csn*1E9, "nF")
#Ahora para Rsn
Dmin = 0.05
Rsn_max = Dmin*Ts/(3*Csn)
print("Rsn_max = ", Rsn_max, "ohm")

Csn_min =  3.692839914747128 nF
Csn =  4.7 nF
Rsn_max =  47.28132387706856 ohm
